# submission 데이터 예측

In [1]:
import pandas as pd
import joblib
import gc

## data load

In [2]:
df = pd.DataFrame()

for i in range(4):
    df = pd.concat([df, pd.read_csv(f'./submission_data_r2/{i}.csv')], axis=0, ignore_index=True)

In [3]:
df.shape

(1674896, 1037)

#### protein one-hot
- 학습시 protein one-hot 정보가 가장 마지막에 들어간다.
- 미리 만들어둔 데이터에 protein one-hot encoding 추가

In [4]:
from sklearn.preprocessing import OneHotEncoder
import gc

ohe = OneHotEncoder(sparse=False)
ohe_re = ohe.fit_transform(df[['protein_name']])

df = pd.concat([df, pd.DataFrame(ohe_re, columns=[col for col in ohe.categories_[0]])], axis=1)

del ohe
del ohe_re
gc.collect()

/opt/anaconda3/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


0

# X data 생성
- 결과 예측 데이터 생성
- X의 형태
    - (1674896, 1038)
    - 'reactivity', 'molecular weight', 'Steric strain'
    - 'LogP', 'TPSA', 'NHBD', 'NHBA', 'planarity'
    - 
    - ecfp -> 0 ~ 1023
    - 'BRD4', 'HSA', 'sEH'

In [5]:
# 예측에 불필요한 feature 제거
X = df.drop(['id', 'protein_name'], axis=1)

# X 형태 확인
# (1674896, 1038)가 나와야됨
X.shape

(1674896, 1038)

In [6]:
X.head()

,reactivity,molecular weight,Steric strain,LogP,TPSA,NHBD,NHBA,planarity,PSA,QED,...,1017,1018,1019,1020,1021,1022,1023,BRD4,HSA,sEH
0,19.029854,615.042,12,4.8093,103.86,4,12,3,103.86,0.171523,...,0,0,0,0,0,0,0,1.0,0.0,0.0
1,19.029854,615.042,12,4.8093,103.86,4,12,3,103.86,0.171523,...,0,0,0,0,0,0,0,0.0,1.0,0.0
2,19.029854,615.042,12,4.8093,103.86,4,12,3,103.86,0.171523,...,0,0,0,0,0,0,0,0.0,0.0,1.0
3,21.611221,689.085,13,2.6868,167.69,5,13,4,167.69,0.103519,...,0,0,0,0,0,0,0,1.0,0.0,0.0
4,21.611221,689.085,13,2.6868,167.69,5,13,4,167.69,0.103519,...,0,0,0,0,0,0,0,0.0,1.0,0.0


# 예측

In [7]:
X.shape

(1674896, 1038)

In [14]:
# model load
XGB_model = joblib.load('../models/voting_models/XGB(800, 0.005, 50, 0.1)_r2_6.joblib')

#xgb_pred = XGB_model.predict(X)
xgb_proba_pred = XGB_model.predict_proba(X)[:, 1] # 확률 예측

# submission format에 맞춰준다.
xgb_proba_pred_df = pd.DataFrame(xgb_proba_pred)
xgb_re_df = df[['id']]
xgb_rd_df = pd.concat([xgb_re_df, xgb_proba_pred_df], axis=1)
xgb_rd_df.columns = ['id', 'binds']

# csv 파일 생성
xgb_rd_df.to_csv('./voting/voting_XGB(800, 0.005, 50, 0.1)_r2_6_pred.csv', index=False)